## Section 1: Setup & Imports

In [ ]:
import sys
sys.path.insert(0, '/Users/shashanklokare/Documents/copilot_agent/trading_agent')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pathlib import Path
import yaml
import logging

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Configure logging
logging.basicConfig(level=logging.INFO)

print("✓ Imports successful")

## Section 2: Load Trading Metrics Module

In [ ]:
from backtest.trading_metrics import (
    TradingMetrics,
    TradeRecord,
    EquityCurve,
    StrategyPerformance,
    StrategyEvaluator,
)

print("✓ Trading metrics module loaded")

## Section 3: Generate Synthetic Test Data

In [ ]:
def generate_synthetic_equity_curve(num_days=252, annual_return=0.15, daily_vol=0.015, num_trades=50):
    """
    Generate synthetic equity curve for demonstration.
    """
    np.random.seed(42)
    
    # Generate daily returns
    daily_drift = annual_return / 252
    daily_returns = np.random.normal(daily_drift, daily_vol, num_days)
    
    # Build equity curve
    initial_equity = 100000
    equity = initial_equity * np.cumprod(1 + daily_returns)
    
    # Generate trade records
    trades = []
    for i in range(num_trades):
        entry_idx = np.random.randint(0, num_days - 20)
        exit_idx = entry_idx + np.random.randint(5, 20)
        
        entry_price = 100 + np.random.normal(0, 5)
        exit_price = entry_price * (1 + np.random.normal(0.001, 0.02))
        
        direction = "LONG" if np.random.random() > 0.5 else "SHORT"
        quantity = 100
        
        if direction == "LONG":
            pnl = (exit_price - entry_price) * quantity - 50
        else:
            pnl = (entry_price - exit_price) * quantity - 50
        
        trades.append(TradeRecord(
            entry_time=pd.Timestamp('2023-01-01') + timedelta(days=entry_idx),
            entry_price=entry_price,
            exit_time=pd.Timestamp('2023-01-01') + timedelta(days=exit_idx),
            exit_price=exit_price,
            direction=direction,
            quantity=quantity,
            entry_cost=25,
            exit_cost=25,
            pnl=pnl,
        ))
    
    timestamps = pd.date_range('2023-01-01', periods=num_days, freq='D')
    daily_pnl = np.diff(equity, prepend=initial_equity)
    cumulative_pnl = np.cumsum(daily_pnl) - initial_equity
    
    # Create regimes (alternating TREND and RANGE)
    regime_series = pd.Series(
        ['TREND' if i % 60 < 40 else 'RANGE' for i in range(num_days)],
        index=timestamps
    )
    
    return EquityCurve(
        timestamps=timestamps,
        equity=equity,
        daily_returns=daily_returns,
        daily_pnl=daily_pnl,
        cumulative_pnl=cumulative_pnl,
        trades=trades,
        regime_series=regime_series,
    )

# Generate test data
eq_curve = generate_synthetic_equity_curve(num_days=252, annual_return=0.18, daily_vol=0.015, num_trades=80)

print(f"✓ Generated equity curve with {len(eq_curve.trades)} trades")
print(f"  Initial equity: ${eq_curve.equity[0]:,.0f}")
print(f"  Final equity: ${eq_curve.final_equity:,.0f}")
print(f"  Total PnL: ${eq_curve.total_pnl:,.0f}")

## Section 4: Compute Comprehensive Trading Metrics

In [ ]:
# Evaluate strategy
evaluator = StrategyEvaluator(risk_free_rate=0.04)
performance = evaluator.evaluate(eq_curve)

print("\n" + "="*60)
print("TRADING METRICS")
print("="*60)
print(f"CAGR:                {performance.cagr:.2%}")
print(f"Max Drawdown:        {performance.max_drawdown:.2%}")
print(f"Sharpe Ratio:        {performance.sharpe_ratio:.2f}")
print(f"Sortino Ratio:       {performance.sortino_ratio:.2f}")
print(f"Calmar Ratio:        {performance.calmar_ratio:.2f}")
print(f"\nTrade Statistics:")
print(f"  Num Trades:        {performance.num_trades}")
print(f"  Win Rate:          {performance.win_rate:.2%}")
print(f"  Profit Factor:     {performance.profit_factor:.2f}")
print(f"  Avg Win:           ${performance.avg_win:,.0f}")
print(f"  Avg Loss:          ${performance.avg_loss:,.0f}")
print(f"  Payoff Ratio:      {performance.payoff_ratio:.2f}")
print(f"\nCosts:")
print(f"  Total Costs:       ${performance.cost_metrics['total_costs']:,.0f}")
print(f"  Cost % of PnL:     {performance.cost_metrics['cost_pct_of_pnl']:.1f}%")
print(f"\nEfficiency:")
print(f"  Turnover (Annual): {performance.turnover:.2f}x")
print("="*60)

## Section 5: Regime-Based Performance Analysis

In [ ]:
print("\n" + "="*60)
print("REGIME BREAKDOWN")
print("="*60)

regime_df = pd.DataFrame(performance.regime_stats).T
print(regime_df)

# Compute regime concentration
regime_pnls = [abs(stats.get('pnl', 0)) for stats in performance.regime_stats.values()]
max_regime_pnl = max(regime_pnls) if regime_pnls else 0
total_pnl = abs(performance.total_pnl)
regime_concentration = (max_regime_pnl / total_pnl) if total_pnl > 0 else 0

print(f"\nRegime Concentration: {regime_concentration:.2%}")
print(f"  (Strategy is diversified across regimes)" if regime_concentration < 0.7 else "  (WARNING: Concentrated in single regime)")
print("="*60)

## Section 6: Monte Carlo Robustness Testing

In [ ]:
from backtest.monte_carlo import MonteCarloTester

# Run trade shuffling Monte Carlo
mc_result = MonteCarloTester.trade_shuffling_mc(
    trades=eq_curve.trades,
    num_iterations=1000,
    metrics_to_compute=['sharpe_ratio', 'max_drawdown', 'final_pnl', 'win_rate']
)

print("\n" + "="*60)
print("MONTE CARLO RESULTS (1000 Trade Shuffles)")
print("="*60)

for metric, stats in mc_result.summary_stats.items():
    print(f"\n{metric.upper()}:")
    print(f"  Mean:    {stats['mean']:.4f}")
    print(f"  Std:     {stats['std']:.4f}")
    print(f"  5th %ile: {mc_result.percentiles[metric][5]:.4f}")
    print(f"  95th %ile: {mc_result.percentiles[metric][95]:.4f}")

print(f"\nProbability Metrics:")
print(f"  Prob Profitable: {mc_result.prob_profitable:.2%}")
print(f"  Prob Positive Sharpe: {mc_result.prob_positive_sharpe:.2%}")
print("="*60)

## Section 7: Visualize Monte Carlo Distributions

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Sharpe distribution
axes[0, 0].hist(mc_result.metrics_distribution['sharpe_ratio'], bins=50, alpha=0.7, color='blue', edgecolor='black')
axes[0, 0].axvline(performance.sharpe_ratio, color='red', linestyle='--', linewidth=2, label='Actual')
axes[0, 0].set_xlabel('Sharpe Ratio')
axes[0, 0].set_title('MC Sharpe Distribution')
axes[0, 0].legend()

# Max Drawdown distribution
axes[0, 1].hist(mc_result.metrics_distribution['max_drawdown'], bins=50, alpha=0.7, color='red', edgecolor='black')
axes[0, 1].axvline(performance.max_drawdown, color='blue', linestyle='--', linewidth=2, label='Actual')
axes[0, 1].set_xlabel('Max Drawdown')
axes[0, 1].set_title('MC Max Drawdown Distribution')
axes[0, 1].legend()

# Final PnL distribution
axes[1, 0].hist(mc_result.metrics_distribution['final_pnl'], bins=50, alpha=0.7, color='green', edgecolor='black')
axes[1, 0].axvline(100000 + eq_curve.total_pnl, color='red', linestyle='--', linewidth=2, label='Actual')
axes[1, 0].set_xlabel('Final Equity')
axes[1, 0].set_title('MC Final Equity Distribution')
axes[1, 0].legend()

# Win rate distribution
axes[1, 1].hist(mc_result.metrics_distribution['win_rate'], bins=50, alpha=0.7, color='purple', edgecolor='black')
axes[1, 1].axvline(performance.win_rate, color='red', linestyle='--', linewidth=2, label='Actual')
axes[1, 1].set_xlabel('Win Rate')
axes[1, 1].set_title('MC Win Rate Distribution')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print("✓ Monte Carlo distributions plotted")

## Section 8: Equity Curve Visualization with Drawdown

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Equity curve
axes[0].plot(eq_curve.timestamps, eq_curve.equity, linewidth=1.5, color='blue', label='Equity')
axes[0].fill_between(eq_curve.timestamps, eq_curve.equity[0], eq_curve.equity, alpha=0.3, color='blue')
axes[0].set_ylabel('Equity ($)')
axes[0].set_title('Strategy Equity Curve')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Daily returns with regime coloring
colors = ['green' if regime == 'TREND' else 'orange' for regime in eq_curve.regime_series]
axes[1].bar(eq_curve.timestamps, eq_curve.daily_returns * 100, color=colors, alpha=0.6, width=0.8)
axes[1].axhline(0, color='black', linestyle='-', linewidth=0.5)
axes[1].set_ylabel('Daily Return (%)')
axes[1].set_title('Daily Returns (Green=TREND, Orange=RANGE)')
axes[1].grid(True, alpha=0.3)

# Drawdown
cummax = np.maximum.accumulate(eq_curve.equity)
drawdown = (eq_curve.equity - cummax) / cummax * 100
axes[2].fill_between(eq_curve.timestamps, drawdown, 0, alpha=0.5, color='red')
axes[2].plot(eq_curve.timestamps, drawdown, color='darkred', linewidth=1)
axes[2].set_ylabel('Drawdown (%)')
axes[2].set_xlabel('Date')
axes[2].set_title(f'Drawdown (Max: {performance.max_drawdown:.2%})')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Equity curve and drawdown visualized")

## Section 9: Strategy Grading System

In [ ]:
from backtest.strategy_grader import StrategyGrader

grader = StrategyGrader()
grade_result = grader.grade(
    performance=performance,
    monte_carlo_result=mc_result,
    parameter_robustness=None,
)

print(grade_result)

## Section 10: Model Metadata & Registry

In [ ]:
from learning.model_metadata import (
    ModelRecord,
    ModelRegistry,
    TrainingMetadata,
    StrategyGradeResult,
)

# Create model record with grades
model = ModelRecord(
    model_id="strategy_demo_v1",
    model_name="Demo Strategy",
    model_type="strategy",
    version="1.0",
    created_date=pd.Timestamp.now(),
    trained_by="evaluation_framework",
    training_metadata=TrainingMetadata(
        train_start_date=pd.Timestamp('2023-01-01'),
        train_end_date=pd.Timestamp('2023-06-30'),
        test_start_date=pd.Timestamp('2023-07-01'),
        test_end_date=pd.Timestamp('2023-12-31'),
        num_train_samples=126,
        num_test_samples=126,
        features_used=['sma_ratio', 'volatility', 'momentum'],
        target_horizon_bars=[5, 20],
        regimes_covered=['TREND', 'RANGE'],
        config_hash='demo_config_hash',
    ),
    strategy_grade=StrategyGradeResult(
        grade=grade_result.grade,
        sharpe=performance.sharpe_ratio,
        max_drawdown=performance.max_drawdown,
        profit_factor=performance.profit_factor,
        cagr=performance.cagr,
        calmar=performance.calmar_ratio,
        win_rate=performance.win_rate,
        regime_concentration=0.65,
        regime_consistency=0.85,
        monte_carlo_prob_profitable=mc_result.prob_profitable,
        parameter_robustness_score=0.82,
        allowed_modes=grade_result.allowed_modes,
        comments=grade_result.comments,
    ),
)

# Register model
registry = ModelRegistry()
registry.register_model(model)

print("✓ Model registered in registry")
print(f"\nModel Metadata:")
print(f"  ID: {model.model_id}")
print(f"  Name: {model.model_name}")
print(f"  Grade: {model.overall_grade}")
print(f"  Allowed Modes: {model.strategy_grade.allowed_modes}")

## Section 11: Summary & Recommendations

In [ ]:
print("\n" + "="*70)
print("STRATEGY EVALUATION SUMMARY")
print("="*70)

print(f"\nGrade Assignment: {grade_result.grade}")
print(f"Allowed Trading Modes: {', '.join(grade_result.allowed_modes)}")

print(f"\nKey Metrics:")
print(f"  • Sharpe Ratio: {performance.sharpe_ratio:.2f} (Grade threshold: A≥1.2, B≥0.8, C≥0.5)")
print(f"  • Max Drawdown: {performance.max_drawdown:.2%} (Grade threshold: A≤25%, B≤30%, C≤40%)")
print(f"  • Profit Factor: {performance.profit_factor:.2f} (Grade threshold: A≥1.4, B≥1.2, C≥1.1)")
print(f"  • CAGR: {performance.cagr:.2%} (Grade threshold: A≥15%, B≥10%, C≥5%)")
print(f"  • Win Rate: {performance.win_rate:.2%}")

print(f"\nRobustness:")
print(f"  • MC Probability Profitable: {mc_result.prob_profitable:.2%}")
print(f"  • Regime Concentration: {regime_concentration:.2%}")
print(f"  • Trades Executed: {performance.num_trades}")

print(f"\nCosts:")
print(f"  • Total Transaction Costs: ${performance.cost_metrics['total_costs']:,.0f}")
print(f"  • Cost as % of PnL: {performance.cost_metrics['cost_pct_of_pnl']:.1f}%")

print(f"\nRecommendation:")
if grade_result.grade in ['A', 'B']:
    mode = grade_result.allowed_modes[0]
    print(f"  ✓ APPROVED for {mode} trading")
    print(f"  This strategy demonstrates robust, risk-adjusted returns")
elif grade_result.grade == 'C':
    print(f"  ⚠ INCUBATION ONLY")
    print(f"  Further optimization and testing recommended")
else:
    print(f"  ✗ REJECTED for live trading")
    print(f"  Strategy does not meet minimum performance standards")

print("\n" + "="*70)